%load_ext autoreload
%autoreload 2

In [1]:
import requests
import aiohttp
import random
import json
import uuid
import asyncio
import os

import numpy as np
from common.models.trial import Solution, Advisor, WrittenStrategy, Trial
from utils.process import process_solution

from dotenv import load_dotenv

In [2]:

# create new experiment
max_participants = 20  # Number of parallel participants in simulation
experiment_type = 'sim_3states_remote_v1'
local = False

load_dotenv()

if local:
    BACKEND_URL = 'http://localhost:5050'
    BACKEND_USER = 'admin'
    BACKEND_PASSWORD = 'admin'
else:
    BACKEND_USER = os.environ['BACKEND_USER']
    BACKEND_PASSWORD = os.environ['BACKEND_PASSWORD']
    BACKEND_URL = os.environ['BACKEND_URL']


In [3]:
url = f"{BACKEND_URL}/admin/config"
auth = (BACKEND_USER, BACKEND_PASSWORD)


payload = json.dumps({
  "active": True,
  "created_at": "2023-10-17T09:57:36.204000",
  "redirect_url": "https://app.prolific.co/submissions/complete",
  "experiment_type": experiment_type,
  "rewrite_previous_data": True,
  "seed": 1,
  "n_generations": 5,
  "n_ai_players": 3,
  "networks_path": "data/24_02_04",
  "n_sessions_per_generation": 16,
  "n_advise_per_session": 5,
  "n_session_tree_replications": 1,
  "conditions": [
    "w_ai",
    "wo_ai"
  ],
  "n_social_learning_blocks": 1,
  "n_social_learning_networks_per_block": 4,
  "n_practice_trials": 2,
  "n_demonstration_trials": 4,
  "simulate_humans": False,
  "social_learning_trials": [
    "observation",
    "repeat",
    "try_yourself"
  ],
  "main_only": True,
  "session_timeout": 0.5,
})
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Basic YWRtaW46YWRtaW4='
}

response = requests.request("POST", url, headers=headers, data=payload, auth=auth)

print(response.text)

{"id":"65ce5a77dc2d0b5767c822ba","active":true,"created_at":"2024-02-15T18:39:51.390000","redirect_url":"https://app.prolific.co/submissions/complete","experiment_type":"sim_3states_remote_v1","rewrite_previous_data":true,"networks_path":"data/24_02_04","seed":1,"n_generations":5,"n_ai_players":3,"n_sessions_per_generation":16,"n_advise_per_session":5,"n_session_tree_replications":1,"conditions":["w_ai","wo_ai"],"n_social_learning_blocks":1,"n_social_learning_networks_per_block":4,"n_practice_trials":2,"n_demonstration_trials":4,"simulate_humans":false,"social_learning_trials":["observation","repeat","try_yourself"],"main_only":true,"session_timeout":0.5,"shared_first_generation":false}


In [4]:
import aiohttp
import time

session = aiohttp.ClientSession()

async def get_trial(prolific_id, experiment_type):
    url = f"{BACKEND_URL}/session/{experiment_type}/{prolific_id}"
    headers = {'Content-Type': 'application/json'}

    start_time = time.time()  # Start the timer

    async with session.get(url, headers=headers) as response:
        response_time = time.time() - start_time  # Calculate the response time

        if response.status == 200:
            try:
                trial = Trial(**await response.json())
                return trial, response_time
            except:
                return None, response_time
        else:
            return None, response_time


async def post_trial(prolific_id, trial_id, body):
    url = f"{BACKEND_URL}/session/{prolific_id}/{trial_id}"
    headers = {'Content-Type': 'application/json'}

    start_time = time.time()  # Start the timer

    if body is not None:
        async with session.post(url, headers=headers, data=body) as response:
            response_time = time.time() - start_time  # Calculate the response time
            return response.status == 200, response_time
    else:
        async with session.post(url, headers=headers) as response:
            response_time = time.time() - start_time  # Calculate the response time
            return response.status == 200, response_time


In [5]:
from pathlib import Path
from common.utils.utils import estimate_solution_score, estimate_average_player_score
from common.models.network import Network


networks_path = Path("../data/24_02_04")
network_data = json.load(open(networks_path / "networks.json"))
solutions_myopic = json.load(open(networks_path / "solution__myopic.json"))
solutions_m1 = json.load(open(networks_path / "machine_solutions" / "0.json"))
solutions_random = json.load(open(networks_path / "solution__random.json"))
networks_by_id = {n["network_id"]: n for n in network_data}
solutions_myopic_by_id = {s["network_id"]: s for s in solutions_myopic}
solutions_m1_by_id = {s["network_id"]: s for s in solutions_m1}
solutions_random_by_id = {s["network_id"]: s for s in solutions_random}


def _get_solution(network_id, solution_type):
    network = networks_by_id[network_id]
    # get the solution for the network
    if solution_type == "myopic":
        solution = solutions_myopic_by_id[network_id]
    elif solution_type == "machine":
        solution = solutions_m1_by_id[network_id]
    elif solution_type == "random":
        solution = solutions_random_by_id[network_id]
    else:
        raise ValueError("Invalid solution type")

    solution['moves'][0] = network['starting_node']
    score = estimate_solution_score(Network(**network), solution['moves'], 10)
    assert score != 100_000, f"Invalid solution score: {score} for solution: {solution_type} and network: {network_id}"

    return Solution(**solution)


def get_solution(network_id, state):
    assert np.absolute(state.sum() - 1) < 0.0001, f"Invalid state: {state}"
    assert np.all(state >= 0), f"Invalid state: {state}"
    s_type_idx = np.random.choice(list(range(len(state))), p=state)
    s_type = ['random', "myopic", "machine"][s_type_idx]
    return _get_solution(network_id, s_type)


def get_solution_evaluation(solution: Solution, network_id):
    # get rewards
    evaluation = process_solution(networks_by_id[network_id], solution.dict())
    return evaluation

In [6]:
p_mypopic_init = 0.3
max_p = 1.
drop_rate = 0.01
incomplete_rate = 0.01

individual_learning_factors = (
    (0.001, [0,0,1]),
    (0.2, [0,1,0]),
    (0.799, [0,0,0])
)

social_learning_factors = {
    'optimal': (
        (0.11, [0,0,1]),
        (0.89, [0,0,0])
    ),
    'myopic': (
        (0.11, [0,1,0]),
        (0.89, [0,0,0])
    )
}

social_learning_strategy = "best"


def scale_state(state, change):
    state = state + change
    state[2] = np.minimum(state[2], max_p)
    state[1] = np.minimum(state[1], np.minimum(1 - state[2], max_p))
    state[0] = 1 - state[2] - state[1]
    assert np.absolute(state.sum() - 1) < 0.0001, f"Invalid state: {state}"
    assert np.all(state >= 0), f"Invalid state: {state}"
    return state


def sample_change(options):
    p = np.array([o[0] for o in options])
    assert np.all(p >= 0), f"Invalid probabilities: {p}"
    assert np.absolute(p.sum() - 1) < 0.0001, f"Invalid probabilities: {p}"
    values = np.array([o[1] for o in options])
    change_idx = np.random.choice(list(range(len(p))), p=p)
    change = values[change_idx]
    return change


def individual_learning(state):
    change = sample_change(individual_learning_factors)
    state = scale_state(state, change)
    return state


def social_learning(state, strategy):
    change = sample_change(social_learning_factors[strategy])
    state = scale_state(state, change)
    return state

def init_state():
    assert p_mypopic_init <= 0.5, f"Invalid p_mypopic_init: {p_mypopic_init}"
    p_mypopic = random.random() * p_mypopic_init * 2
    state = np.array([1 - p_mypopic, p_mypopic, 0]) # [random, myopic, machine]
    return state

def handle_instruction_trial(trial, state):
    body = None
    return body, state

def handle_individual_trial(trial, state):
    solution = get_solution(trial.network.network_id, state)
    state = individual_learning(state)
    return solution.json(), state

def handle_written_strategy_trial(trial, state):
    strategy = WrittenStrategy(
        strategy=''
    )
    body = strategy.json()
    return body, state

def handle_demonstration_trial(trial, state):
    solution = get_solution(trial.network.network_id, state)
    if random.random() < incomplete_rate:
        solution.moves = solution.moves[:-random.randint(1, 3)]
    return solution.json(), state

def handle_debriefing_trial(trial, state):
    body = None
    return body, state

def handle_social_learning_selection_trial(trial, state):
    advisor_selection = trial.advisor_selection
    advisor = advisor_selection.advisor_ids
    scores = advisor_selection.scores

    if social_learning_strategy == "best":
        # select the advisor with the highest score
        max_score_idx = np.argmax(scores)
        advisor = advisor[max_score_idx]
    elif social_learning_strategy == "random":
        # select a random advisor
        advisor = random.choice(advisor)
    else:
        raise ValueError(f"Invalid social learning strategy: {social_learning_strategy}")

    selection = Advisor(
        advisor_id=advisor
    )
    body = selection.json()
    return body, state

def handle_observation_trial(trial, state):
    solution = trial.advisor.solution
    network_id = trial.network.network_id
    evaluation = get_solution_evaluation(solution, network_id)
    body = None
    return body, state

def handle_repeat_trial(trial, state):
    solution = trial.advisor.solution

    solution = trial.advisor.solution
    network_id = trial.network.network_id
    evaluation = get_solution_evaluation(solution, network_id)
    if evaluation['optimal'] == 10:
        state = social_learning(state, 'optimal')
    elif evaluation['myopic'] > 0:
        state = social_learning(state, 'myopic')

    body = solution.json()
    return body, state

def handle_try_yourself_trial(trial, state):
    solution = get_solution(trial.network.network_id, state)
    state = individual_learning(state)
    return solution.json(), state


def handle_trial(trial, state):
    if trial.trial_type == "instruction":
        return handle_instruction_trial(trial, state)
    elif trial.trial_type == "individual":
        return handle_individual_trial(trial, state)
    elif trial.trial_type == "written_strategy":
        return handle_written_strategy_trial(trial, state)
    elif trial.trial_type == "demonstration":
        return handle_demonstration_trial(trial, state)
    elif trial.trial_type == "debriefing":
        return handle_debriefing_trial(trial, state)
    elif trial.trial_type == "social_learning_selection":
        return handle_social_learning_selection_trial(trial, state)
    elif trial.trial_type == "observation":
        return handle_observation_trial(trial, state)
    elif trial.trial_type == "repeat":
        return handle_repeat_trial(trial, state)
    elif trial.trial_type == "try_yourself":
        return handle_try_yourself_trial(trial, state)
    else:
        raise ValueError(f"{trial.trial_type} is an invalid trial type")

In [7]:

async def run_participant():
    max_skip = 10
    trials = []
    prolific_id = "sim_" + uuid.uuid4().hex[:8]
    state = init_state()
    current_trial_id = None
    while True:
        trial, get_response_time = await get_trial(prolific_id, experiment_type)
        if trial is None:
            await asyncio.sleep(5)
            max_skip -= 1
            if max_skip < 0:
                break
            continue
        if trial.id == current_trial_id:
            if trial.trial_type == "debriefing":
                break
            else:
                raise ValueError(f"Trial {trial.id} of type {trial.trial_type} of prolific id {prolific_id} is a duplicate")

        if random.random() < drop_rate / 2:
            trials = [{**t, 'dropped': True} for t in trials]
            break

        current_trial_id = trial.id
        body, state = handle_trial(trial, state)
        _, post_response_time = await post_trial(prolific_id, trial.id, body)
        trial_clean = json.loads(trial.json())
        # session_clean = json.loads(trail.session.json())
        trials.append({'trial': trial_clean, 'prolific_id': prolific_id, 'response_time': get_response_time, 'post_response_time': post_response_time})

        if random.random() < drop_rate / 2:
            trials = [{**t, 'dropped': True} for t in trials]
            break

    return trials

In [8]:

def check_uncompleted(experiment_type):
    completed = False
    url = f'{BACKEND_URL}/results'
    headers = {'Accept': 'application/json'}
    auth = (BACKEND_USER, BACKEND_PASSWORD)
    sessions = requests.get(f'{url}/sessions?experiment_type={experiment_type}&completed={completed}', headers=headers, auth=auth)
    sessions_json = sessions.json()

    n_uncompleted_sessions = len([s for s in sessions_json if (not s['finished'] and not s['expired'])])
    return n_uncompleted_sessions


async def run_with_limit(semaphore, trials):
    async with semaphore:
        new_trials = await run_participant()
        if new_trials:
            trials.extend(new_trials)


async def main(max_concurrent_tasks):
    trials = []
    semaphore = asyncio.Semaphore(max_concurrent_tasks)
    tasks = []

    n_loop = 0
    while True:
        n_sessions = check_uncompleted(experiment_type) - len(tasks)
        if n_sessions == 0:
            break

        n_completed = 0
        n_started = 0

        # Assuming you have a way to determine if more participants should be run
        while n_completed < n_sessions:
            while len(tasks) < max_concurrent_tasks and n_started < n_sessions:
                print(f"Loop {n_loop}, Started: {n_started}, Finished: {n_completed}, Remaining: {n_sessions - n_completed}, Running: {len(tasks)}, Total: {n_sessions}")
                task = asyncio.create_task(run_with_limit(semaphore, trials))
                n_started += 1
                tasks.append(task)

            # Wait for one of the tasks to complete
            done, _ = await asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED)

            n_completed += len(done)

            # Remove the completed tasks
            tasks = [t for t in tasks if not t.done()]


        n_loop += 1
    # Wait for the remaining tasks to complete
    if tasks:
        await asyncio.wait(tasks)

    return trials

# Run the main coroutine with the desired maximum number of concurrent tasks

trials = await main(max_participants)

# Save trials as json
with open('trials.json', 'w') as f:
    json.dump(trials, f, indent=4)


Loop 0, Started: 0, Finished: 0, Remaining: 77, Running: 0, Total: 77
Loop 0, Started: 1, Finished: 0, Remaining: 77, Running: 1, Total: 77
Loop 0, Started: 2, Finished: 0, Remaining: 77, Running: 2, Total: 77
Loop 0, Started: 3, Finished: 0, Remaining: 77, Running: 3, Total: 77
Loop 0, Started: 4, Finished: 0, Remaining: 77, Running: 4, Total: 77
Loop 0, Started: 5, Finished: 0, Remaining: 77, Running: 5, Total: 77
Loop 0, Started: 6, Finished: 0, Remaining: 77, Running: 6, Total: 77
Loop 0, Started: 7, Finished: 0, Remaining: 77, Running: 7, Total: 77
Loop 0, Started: 8, Finished: 0, Remaining: 77, Running: 8, Total: 77
Loop 0, Started: 9, Finished: 0, Remaining: 77, Running: 9, Total: 77
Loop 0, Started: 10, Finished: 0, Remaining: 77, Running: 10, Total: 77
Loop 0, Started: 11, Finished: 0, Remaining: 77, Running: 11, Total: 77
Loop 0, Started: 12, Finished: 0, Remaining: 77, Running: 12, Total: 77
Loop 0, Started: 13, Finished: 0, Remaining: 77, Running: 13, Total: 77
Loop 0, Star

Task exception was never retrieved
future: <Task finished name='Task-17' coro=<run_with_limit() done, defined at /tmp/ipykernel_1893365/3014096551.py:13> exception=TypeError('cannot unpack non-iterable bool object')>
Traceback (most recent call last):
  File "/tmp/ipykernel_1893365/3014096551.py", line 15, in run_with_limit
    new_trials = await run_participant()
  File "/tmp/ipykernel_1893365/2370532245.py", line 27, in run_participant
    _, post_response_time = await post_trial(prolific_id, trial.id, body)
TypeError: cannot unpack non-iterable bool object
Task exception was never retrieved
future: <Task finished name='Task-8' coro=<run_with_limit() done, defined at /tmp/ipykernel_1893365/3014096551.py:13> exception=TypeError('cannot unpack non-iterable bool object')>
Traceback (most recent call last):
  File "/tmp/ipykernel_1893365/3014096551.py", line 15, in run_with_limit
    new_trials = await run_participant()
  File "/tmp/ipykernel_1893365/2370532245.py", line 27, in run_parti

Loop 0, Started: 20, Finished: 1, Remaining: 76, Running: 19, Total: 77
Loop 0, Started: 21, Finished: 2, Remaining: 75, Running: 19, Total: 77
Loop 0, Started: 22, Finished: 3, Remaining: 74, Running: 19, Total: 77


Task exception was never retrieved
future: <Task finished name='Task-21' coro=<run_with_limit() done, defined at /tmp/ipykernel_1893365/3014096551.py:13> exception=TypeError('cannot unpack non-iterable bool object')>
Traceback (most recent call last):
  File "/tmp/ipykernel_1893365/3014096551.py", line 15, in run_with_limit
    new_trials = await run_participant()
  File "/tmp/ipykernel_1893365/2370532245.py", line 27, in run_participant
    _, post_response_time = await post_trial(prolific_id, trial.id, body)
TypeError: cannot unpack non-iterable bool object
Task exception was never retrieved
future: <Task finished name='Task-6' coro=<run_with_limit() done, defined at /tmp/ipykernel_1893365/3014096551.py:13> exception=TypeError('cannot unpack non-iterable bool object')>
Traceback (most recent call last):
  File "/tmp/ipykernel_1893365/3014096551.py", line 15, in run_with_limit
    new_trials = await run_participant()
  File "/tmp/ipykernel_1893365/2370532245.py", line 27, in run_parti

Loop 0, Started: 23, Finished: 4, Remaining: 73, Running: 19, Total: 77
Loop 0, Started: 24, Finished: 5, Remaining: 72, Running: 19, Total: 77
Loop 0, Started: 25, Finished: 6, Remaining: 71, Running: 19, Total: 77
Loop 0, Started: 26, Finished: 7, Remaining: 70, Running: 19, Total: 77
Loop 0, Started: 27, Finished: 8, Remaining: 69, Running: 19, Total: 77
Loop 0, Started: 28, Finished: 10, Remaining: 67, Running: 18, Total: 77
Loop 0, Started: 29, Finished: 10, Remaining: 67, Running: 19, Total: 77
Loop 0, Started: 30, Finished: 11, Remaining: 66, Running: 19, Total: 77
Loop 0, Started: 31, Finished: 12, Remaining: 65, Running: 19, Total: 77
Loop 0, Started: 32, Finished: 13, Remaining: 64, Running: 19, Total: 77


CancelledError: 